In [ ]:
import pandas as pd
import spacy
from spacy import displacy
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet
#stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Helper functions

In [ ]:
def get_root(phrase):
  for token in phrase:
    if token.dep_ == 'ROOT':
      return token

In [ ]:
#takes a phrase as input in the tokenized fromat
#breadth-first search of the dependency tree of the given phrase 
#returns a the dictionary with the length of the path to root for each token in the sentence
def len_path_root(phrase):
  dist = 0
  root_token = get_root(phrase)
  children = list(root_token.children)
  lengths = {
      root_token: dist
  }
  children_count = len(list(children))
  while children_count != 0:
    dist += 1
    new_children = []
    for token in children:
      lengths[token] = dist
      new_children.extend(token.children)
    children=new_children
    children_count = len(list(new_children))
  
  #assign length 3 (chosen randomly) for nodes not connected to the root 
  for token in phrase:
    if token not in lengths:
      lengths[token] = 3
  return lengths

In [ ]:
def join_punctuation(seq, characters='.,;?!'):
    characters = set(characters)
    seq = iter(seq)
    current = next(seq)

    for nxt in seq:
        if nxt in characters:
            current += nxt
        else:
            yield current
            current = nxt

    yield current

In [ ]:
#takes the dataset, phrase_id and chapter_id
#retruns the phrase in a string by merging the words
#used to iterate in the training and testing set for extracting phrases 

def get_text(table, id,chapter):
  phrase_table = table[(table['phrase_id'] == id) & (table['chapter_id'] == chapter)]
  return ' '.join(join_punctuation(phrase_table['word'].values))

# def get_text(table, id,chapter):
#   phrase_table = table[(table['phrase_id'] == id) & (table['chapter_id'] == chapter)]
#   return ' '.join(phrase_table['word'].values)

In [ ]:
#same as above
#returns a list with the labels of a phrase identified with the ch_id, and phr_id
def get_labels(table, id,chapter):
  phrase_table = table[(table['phrase_id'] == id) & (table['chapter_id'] == chapter)]
  return phrase_table['label'].values

In [ ]:
#adjusted lemmatization for nltk library
#offers the POS as a parameter to lemmatization function to make it more precise
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
#used to match the length of the tokenization with the length of the filtered table 
def tokenize(arg, ch = 'baskervilles03', ph = 21):

  if (ph, ch) == (436, 'wisteria02'):
    with arg.retokenize() as retokenizer:
      retokenizer.merge(arg[0:2])
      retokenizer.merge(arg[4:7])
    return arg

  no_exc = [('baskervilles03', 16), ('baskervilles03', 20), ('baskervilles11', 45), ('baskervilles12', 283), ('baskervilles13', 271), ('baskervilles14', 55)]
  retok1_pos = []#for - 
  retok2_pos = []#for `

  #1
  shift = 0
  cr_pos = 0
  for token in arg:
    if token.text == '-':
      retok1_pos.append(cr_pos)
    cr_pos+=1
    prev_char = token.text
  
  for pos in retok1_pos:
    with arg.retokenize() as retokenizer:
      retokenizer.merge(arg[pos-1-shift:pos+2-shift])
      shift += 2 

  #2
  shift = 0
  cr_pos = 0
  prev_char = 0
  for token in arg:
    if token.text =='`' and prev_char == '`':
      retok2_pos.append(cr_pos)
    cr_pos+=1
    prev_char = token.text
 
  for pos in retok2_pos:
    with arg.retokenize() as retokenizer:
      retokenizer.merge(arg[pos-shift-1:pos+1-shift])
      shift += 1

  #3
  retok2_pos = []
  suf = ['66', '86','ve','m']
  shift = 0
  cr_pos = 0
  prev_char = 0
  for token in arg:
    if token.text in suf and prev_char == "'" or token.text == '.' and prev_char == "No" and (ch,ph) not in no_exc:
      retok2_pos.append(cr_pos)
    cr_pos+=1
    prev_char = token.text
 
  for pos in retok2_pos:
    with arg.retokenize() as retokenizer:
      retokenizer.merge(arg[pos-shift-1:pos+1-shift])
      shift += 1

  if ph in [0,'0']:
    with arg.retokenize() as retokenizer:
      retokenizer.merge(arg[2:4])

  
  return arg

## [Not needed anymore] Testing attributes on individual pre-set phrase before automatically adding to the table dataset

In [ ]:
#testing if tokenize and data from table have the same length for each phrase
all_ch_ids = train_data['chapter_id'].unique()
mismatch=[]
for ch in all_ch_ids:
  filter_ch = train_data[train_data['chapter_id'] == ch]
  all_ph_ids = filter_ch['phrase_id'].unique()
  for ph in all_ph_ids:
    filter_ph = filter_ch[filter_ch['phrase_id'] == ph]
    phrase = get_text(ph, ch)
    phr_doc = nlp(phrase)
    tok = tokenize(phr_doc, ch, ph)
    if len(filter_ph) != len(tok):
      mismatch.append((ch,ph))
print(len(mismatch))

0


In [ ]:
#testing if there is any negation in 
all_ch_ids = train_data['chapter_id'].unique()
mismatch=[]
for ch in all_ch_ids:
  filter_ch = train_data[train_data['chapter_id'] == ch]
  all_ph_ids = filter_ch['phrase_id'].unique()
  for ph in all_ph_ids:
    filter_ph = filter_ch[filter_ch['phrase_id'] == ph]
    phrase = get_text(ph, ch)
    phr_doc = nlp(phrase)
    tok = tokenize(phr_doc, ch, ph)
    if len(filter_ph) != len(tok):
      mismatch.append((ch,ph))
print(len(mismatch))

In [ ]:
#errors
phrase = get_text(436, 'wisteria02')
phr_doc = nlp(phrase)
#toks = tokenize(phr_doc,436)
for tok in phr_doc:
  print(tok,'\n')

# flag = 0
# if(all(x in abc for x in mismatch)):
#     flag = 1
# print(flag)

In [ ]:
#nbor(d) - neighbour in the initial sentence at distance d -/+ -> to left/right
phr1 = "He is interested in learning Natural Language Processing."
phr2 = "I stood upon the hearth-rug and picked up the stick which our visitor had left behind him the night before."
phr3 = "Gus Proto is a Python developer currently working for a London-based Fintech company"

phr_doc = nlp(phr2)
res = len_path_root(phr_doc)

for token in phr_doc:
  print(token.text, res[token], "\n")

displacy.render(phr_doc, style="dep", jupyter=True)

In [ ]:
phr4 = "Mr. Sherlock Holmes , who was usually very late in the mornings , save upon those not infrequent occasions when he was up all night , was seated at the breakfast table ."
phr4 = "guru99 is a totally new kind of learning experience."
phr4 = "The striped bats are hanging on their feet for best"

#phr4 = sent_tokenize(phr4)
words_list = nltk.word_tokenize(phr4)
print(tokenize(words_list))
print(words_list)
#adjusted lemma
#lemmatizer = WordNetLemmatizer()
#print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words_list])

#POS 1,2
# fine_tags = nltk.pos_tag(words_list)
# coarse_tags = nltk.pos_tag(words_list, tagset='universal')
# print(fine_tags)
# print(coarse_tags)

In [ ]:
id = 15
ch = 'baskervilles01'
txt = get_text(id,ch)
txt = "I wouldn't do that"
phr_doc = nlp(txt)

#print(len(list(phr_doc)))

# with phr_doc.retokenize() as retokenizer:
#     retokenizer.merge(phr_doc[21:23])

#phr_doc = re_tokenize(phr_doc)
for token in phr_doc:
  print(token)

#displacy.render(phr_doc, style="dep", jupyter=True)

## [Not needed anymore] Clean CRF

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


True

In [ ]:
nltk.corpus.conll2002.fileids()
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))

In [ ]:
print(train_sents[2])

In [ ]:
y_train = [sent2labels(s) for s in train_sents]
print(y_train)

[['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'], ['O'], ['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O'], ['O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O'

## Merging datasets

In [ ]:
#merge datasets
#test datasets
test_card = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ATM/SEM-2012-test-cardboard.txt', sep="\t", header = None)
test_circ = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ATM/SEM-2012-test-circle.txt', sep="\t", header = None)

frames = [test_card, test_circ]
test_data = pd.concat(frames)
test_data.rename(columns={1: 'phrase_id', 0: 'chapter_id', 2:'word_id', 3:'word', 4:'label'}, inplace=True)
#print(test_data.head(40))

#train datasets
train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ATM/SEM-2012-training.txt', sep="\t", header = None)
dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ATM/SEM-2012-dev.txt', sep="\t", header = None)

frames2 = [train, dev]
train_data = pd.concat(frames2)
train_data.rename(columns={1: 'phrase_id', 0: 'chapter_id', 2:'word_id', 3:'word', 4:'label'}, inplace=True)

#merged datasets in train_data, test_data
print(train_data.head(10))

       chapter_id  phrase_id  word_id      word label
0  baskervilles01          0        0   Chapter     O
1  baskervilles01          0        1        1.     O
2  baskervilles01          0        2       Mr.     O
3  baskervilles01          0        3  Sherlock     O
4  baskervilles01          0        4    Holmes     O
5  baskervilles01          1        0       Mr.     O
6  baskervilles01          1        1  Sherlock     O
7  baskervilles01          1        2    Holmes     O
8  baskervilles01          1        3         ,     O
9  baskervilles01          1        4       who     O


## Dataset exploration

In [ ]:
all_ch_ids = train_data['chapter_id'].unique()
print(all_ch_ids)

['baskervilles01' 'baskervilles02' 'baskervilles03' 'baskervilles04'
 'baskervilles05' 'baskervilles06' 'baskervilles07' 'baskervilles08'
 'baskervilles09' 'baskervilles10' 'baskervilles11' 'baskervilles12'
 'baskervilles13' 'baskervilles14' 'wisteria01' 'wisteria02']


In [ ]:
all_ch_ids = train_data['chapter_id'].unique()
all_features = []
all_labels = []
for ch in all_ch_ids:
  filter_ch = train_data[train_data['chapter_id'] == ch]
  #print(len(filter_ch))
  all_ph_ids = filter_ch['phrase_id'].unique()
  print(len(all_ph_ids))

156
165
249
287
280
200
342
134
399
251
295
316
299
271
347
440


In [ ]:
#num chapters training
phrase_lengths = []

all_ch_ids = train_data['chapter_id'].unique()
all_features = []
all_labels = []
for ch in all_ch_ids:
  filter_ch = train_data[train_data['chapter_id'] == ch]
  all_ph_ids = filter_ch['phrase_id'].unique()
  for ph in all_ph_ids:
    filter_ph = filter_ch[filter_ch['phrase_id'] == ph]
    phrase_lengths.append(len(filter_ph))

print(max(phrase_lengths))
print(min(phrase_lengths))
print(sum(phrase_lengths) / len(phrase_lengths))

83
2
17.83299480929813


In [ ]:
print(len(train_data))
print(len(test_data))

79018
19216


In [ ]:
all_ch_ids = test_data['chapter_id'].unique()
print(all_ch_ids)

['cardboard' 'circle01' 'circle02']


In [ ]:
all_ch_ids = test_data['chapter_id'].unique()
all_features = []
all_labels = []
for ch in all_ch_ids:
  filter_ch = test_data[test_data['chapter_id'] == ch]
  #print(len(filter_ch))
  all_ph_ids = filter_ch['phrase_id'].unique()
  print(len(all_ph_ids))

496
371
222


In [ ]:
phrase_lengths = []

all_ch_ids = test_data['chapter_id'].unique()
all_features = []
all_labels = []
for ch in all_ch_ids:
  filter_ch = test_data[test_data['chapter_id'] == ch]
  all_ph_ids = filter_ch['phrase_id'].unique()
  for ph in all_ph_ids:
    filter_ph = filter_ch[filter_ch['phrase_id'] == ph]
    phrase_lengths.append(len(filter_ph))

print(max(phrase_lengths))
print(min(phrase_lengths))
print(sum(phrase_lengths) / len(phrase_lengths))

68
2
17.6455463728191


In [ ]:
print(list(train_data['label'].values).count('B-NEG'))
print(list(train_data['label'].values).count('O'))
print(list(test_data['label'].values).count('B-NEG'))
print(list(test_data['label'].values).count('O'))

1163
77836
269
18942


## CRF functions

In [ ]:
def word2feature(token):

  features = {
    'lemma':token.lemma_,
    'fine_pos': token.pos_,
    'coarse_pos': token.tag_,
    'dependency':token.dep_,
    'head':token.head
  }

  return features

In [ ]:
#takes a token as input
#returns True if token should be kept, or False if it is filtered
#could be changed depending on performance

neg_list = ['nor', 'neither', 'without', 'nobody', 'none', 'nothing', 'never', 'not', 'no', 'nowhere', 'non', "n't", "rather", "than", 'for', 'the']
def keep(tok):
  if tok.text in neg_list:
    return True
  if tok.is_punct or tok.is_stop:
    return False
  

In [ ]:
#error on cardboard 450
#takes as input text of a sentence
#returns a list of dictionaries with the features of its tokens
# def sent2feature(sentence, labels, ch = 'baskervilles03', ph = 21):
#   sent_feat = []
#   lemmatizer = WordNetLemmatizer()
#   sent_doc = nlp(sentence)
#   tokens = tokenize(sent_doc, ch, ph)
#   lengths = len_path_root(tokens)

#   ord = 0
#   shift = 0
#   for tok in tokens:

#     if keep(tok):
#       features = word2feature(tok)
#       features['len_path_root'] = lengths[tok]
#       sent_feat.append(features)

#     else:
#       labels = np.delete(labels, ord-shift)
#       shift+=1

#     ord+=1

#   return sent_feat, labels

In [ ]:
#error on cardboard 450
#takes the table as an input
#returns the list of lists of dicitionaries with the features
#text->phrase->words 

# def process_data(table):
#   all_ch_ids = table['chapter_id'].unique()
#   all_features = []
#   all_labels = []
#   for ch in all_ch_ids:
#     filter_ch = table[table['chapter_id'] == ch]
#     all_ph_ids = filter_ch['phrase_id'].unique()
#     for ph in all_ph_ids:
#       phrase = get_text(table, ph, ch)
#       labels = get_labels(table, ph, ch)
#       #filtered
#       print(ch,ph)
#       filt_features, filt_labels = sent2feature(phrase, labels, ch, ph)
#       all_features.append(filt_features)
#       all_labels.append(filt_labels)

#   return all_features, all_labels

In [ ]:
def sent2feature(sentence, ch = 'baskervilles03', ph = 21):
  sent_feat = []
  lemmatizer = WordNetLemmatizer()
  sent_doc = nlp(sentence)
  tokens = tokenize(sent_doc, ch, ph)
  lengths = len_path_root(tokens)

  ord = 0
  shift = 0
  for tok in tokens:
    features = word2feature(tok)
    features['len_path_root'] = lengths[tok]
    sent_feat.append(features)

  return sent_feat

In [ ]:
#working
#takes the table as an input
#returns the list of lists of dicitionaries with the features
#text->phrase->words 

def process_data(table):
  all_ch_ids = table['chapter_id'].unique()
  all_features = []
  all_labels = []
  for ch in all_ch_ids:
    filter_ch = table[table['chapter_id'] == ch]
    all_ph_ids = filter_ch['phrase_id'].unique()
    for ph in all_ph_ids:
      filter_ph = filter_ch[filter_ch['phrase_id'] == ph]
      phrase = get_text(table, ph, ch)
      labels = get_labels(table, ph, ch)
      all_features.append(sent2feature(phrase, ch, ph))
      all_labels.append(labels)

  return all_features, all_labels

In [ ]:
x_train, y_train = process_data(train_data)
x_test, y_test = process_data(test_data)

In [ ]:
print(y_train[53])

['O' 'O' 'B-NEG' 'O' 'O' 'O']


In [ ]:
doc = nlp("I don't like apples and pasta.")
for tok in doc:
  print(tok.text, tok.is_stop, '\n')

In [ ]:
import spacy
from spacy import displacy
from pathlib import Path

phr1 = get_text(train_data, 8, 'baskervilles01')
print(phr1)
doc = nlp(phr1)
svg = displacy.render(doc, style="dep", jupyter = True)

# output_path = Path("dependency_plot.svg") # you can keep there only "dependency_plot.svg" if you want to save it in the same folder where you run the script 
# output_path.open("w", encoding="utf-8").write(svg)

Holmes was sitting with his back to me, and I had given him no sign of my occupation.


In [ ]:
feat_final = sent2feature(doc, ch = 'baskervilles03', ph = 21)
print(feat_final)
# for dictt in feat_final:
#   print(dictt, '\n')

[{'lemma': 'Holmes', 'fine_pos': 'PROPN', 'coarse_pos': 'NNP', 'dependency': 'nsubj', 'head': sitting, 'len_path_root': 1}, {'lemma': 'be', 'fine_pos': 'AUX', 'coarse_pos': 'VBD', 'dependency': 'aux', 'head': sitting, 'len_path_root': 1}, {'lemma': 'sit', 'fine_pos': 'VERB', 'coarse_pos': 'VBG', 'dependency': 'ROOT', 'head': sitting, 'len_path_root': 0}, {'lemma': 'with', 'fine_pos': 'ADP', 'coarse_pos': 'IN', 'dependency': 'prep', 'head': sitting, 'len_path_root': 1}, {'lemma': 'his', 'fine_pos': 'PRON', 'coarse_pos': 'PRP$', 'dependency': 'poss', 'head': back, 'len_path_root': 3}, {'lemma': 'back', 'fine_pos': 'NOUN', 'coarse_pos': 'NN', 'dependency': 'pobj', 'head': with, 'len_path_root': 2}, {'lemma': 'to', 'fine_pos': 'ADP', 'coarse_pos': 'IN', 'dependency': 'prep', 'head': back, 'len_path_root': 3}, {'lemma': 'I', 'fine_pos': 'PRON', 'coarse_pos': 'PRP', 'dependency': 'pobj', 'head': to, 'len_path_root': 4}, {'lemma': ',', 'fine_pos': 'PUNCT', 'coarse_pos': ',', 'dependency': 'pu

In [ ]:
dictio = {
    'lemma': 'back', 
    'fine_pos': 'NOUN', 
    'coarse_pos': 'NN', 
    'dependency': 'pobj', 
    'head': 'with', 
    'len_path_root': 2,
    'previous_word':'his',
    'next_word':'to',
    'affix': 0
} 

for key in dictio:
  print(key,':',dictio[key] ,'\n')


lemma : back 

fine_pos : NOUN 

coarse_pos : NN 

dependency : pobj 

head : with 

len_path_root : 2 

previous_word : his 

next_word : to 

affix : 0 

